In [ ]:
%pip install hazm

In [3]:
from __future__ import unicode_literals
from hazm import *
from tqdm import tqdm
import json
with open("./IR_data_news_12k.json", "r") as read_file:
    data = json.load(read_file)

In [45]:
def print_result(docs):
  for doc in docs:
    print(data[doc]['title'], data[doc]['url'], '----------------------------')

In [4]:
normalizer = Normalizer()
lemmatizer = Lemmatizer()


def preprocess(text):
    text = normalizer.normalize(text)
    # print(text)
    words = word_tokenize(text)
    # print(words)
    stop_words = stopwords_list()
    words[:] = [x for x in words if not (x in stop_words)]

    for i in range(len(words)):
        words[i] = lemmatizer.lemmatize(words[i])

    return words


for doc, value in tqdm(data.items()):
    data[doc]['title'] = preprocess(data[doc]['title'])
    data[doc]['content'] = preprocess(data[doc]['content'])

100%|██████████| 12202/12202 [01:02<00:00, 195.91it/s]


In [5]:
print(data["0"])

{'title': ['اعلام', 'زمان', 'قرعه', 'کشید#کش', 'جام', 'باشگاه', 'فوتسال', 'آسیا'], 'content': ['گزارش', 'خبرگزاری', 'فارس', '،', 'کنفدراسیون', 'فوتبال', 'آسیا', '(', 'AFC', ')', 'نامه', 'رسم', 'فدراسیون', 'فوتبال', 'ایران', 'باشگاه', 'گیتی', 'پسند', 'زمان', 'قرعه', 'کشید#کش', 'جام', 'باشگاه', 'فوتسال', 'آسیا', 'رسما', 'اعلام', '.', 'اساس', '۲۵', 'فروردین', 'ماه', '۱۴۰۱', 'مراسم', 'قرعه', 'کشید#کش', 'جام', 'باشگاه', 'فوتسال', 'آسیا', 'مالزی', 'برگزار', '.', 'باشگاه', 'گیتی', 'پسند', 'بعنوان', 'قهرمان', 'فوتسال', 'ایران', 'سال', '۱۴۰۰', 'مسابقات', '.', 'گیتی', 'پسند', 'تجربه', '۳', 'دوره', 'حضور', 'جام', 'باشگاه', 'فوتسال', 'آسیا', 'دوره', 'فینال', 'مسابقات', 'عنوان', 'قهرمان', 'مقام', 'دوم', 'بدست', 'آورد#آور', '.', 'انتهای', 'پیام/'], 'tags': ['اعلام زمان', 'قرعه\u200cکشی', 'قرعه\u200cکشی جام', 'قرعه\u200cکشی جام باشگاه\u200cهای فوتسال', 'ای اف سی', 'گیتی پسند'], 'date': '3/15/2022 5:59:27 PM', 'url': 'https://www.farsnews.ir/news/14001224001005/اعلام-زمان-قرعه-کشی-جام-باشگاه-های-فوتسا

In [6]:
title_pos_index = {}
content_pos_index = {}

for doc in data:
    for pos, term in enumerate(data[doc]["title"]):
        if term in title_pos_index:
            title_pos_index[term][0] = title_pos_index[term][0] + 1

            if doc in title_pos_index[term][1]:
                title_pos_index[term][1][doc].append(pos)
                    
            else:
                title_pos_index[term][1][doc] = [pos]
        else:
            title_pos_index[term] = []
            title_pos_index[term].append(1)
            title_pos_index[term].append({})     
            title_pos_index[term][1][doc] = [pos]
        
    for pos, term in enumerate(data[doc]["content"]):
        if term in content_pos_index:
            content_pos_index[term][0] = content_pos_index[term][0] + 1

            if doc in content_pos_index[term][1]:
                content_pos_index[term][1][doc].append(pos)
                    
            else:
                content_pos_index[term][1][doc] = [pos]
        else:
            content_pos_index[term] = []
            content_pos_index[term].append(1)
            content_pos_index[term].append({})     
            content_pos_index[term][1][doc] = [pos]

In [7]:
print(title_pos_index["سلام"])
print(content_pos_index["سلام"])

[2, {'3413': [4], '6967': [1]}]
[364, {'314': [82], '860': [183], '964': [300], '1102': [90], '1221': [80], '1312': [82], '1322': [708], '1382': [61, 70, 74, 79], '1585': [53], '1858': [19], '1903': [115], '1961': [20], '1963': [22], '2046': [45], '2276': [260], '2313': [36], '2563': [281], '2617': [25], '3422': [146], '3509': [34, 40], '3537': [157], '3662': [294], '4005': [60, 63], '4033': [100], '4120': [38], '4152': [157], '4169': [105], '4244': [65], '4455': [98], '4592': [34], '4607': [61], '4873': [272], '5068': [42], '5116': [18], '5544': [168], '5800': [76], '6675': [94], '6847': [727], '6895': [72], '6915': [1956, 2193], '6967': [13, 44], '6988': [38], '7016': [164], '7018': [136], '7087': [32], '7143': [42], '7181': [44], '7219': [55], '7231': [38], '7234': [48], '7241': [24], '7253': [107], '7259': [505, 510], '7260': [233, 236], '7272': [286], '7331': [132], '7334': [53], '7423': [125], '7517': [60], '7541': [32], '7554': [186], '7575': [55], '7579': [112], '7694': [48], '

In [8]:
def boolean_include_query(term):
  candidates = content_pos_index[term][1]
  return [c[0] for c in sorted(candidates.items(), key=lambda x: len(x[1]), reverse=True)]

def boolean_exclude_query(term, candidates):
  exclude = boolean_include_query(term)
  return [x for x in candidates if not (x in exclude)]

def candidates_intersection(candidates):
  res = candidates[0]
  for i in range(1, len(candidates)):
    res = [value for value in res if value in candidates[i]]
  return res
    
def boolean_query(query):
  query_words = preprocess(query)

  to_exclude = []
  candidates = []

  for index in range(len(query_words)):
    if query_words[index] == "!":
      index = index + 1
      to_exclude.append(query_words[index])
    elif index == 0 or query_words[index-1] != "!":
      candidates.append(boolean_include_query(query_words[index]))

  candidates = candidates_intersection(candidates)
  for word in to_exclude:
    candidates = boolean_exclude_query(word, candidates)
  
  return candidates
  

In [24]:
def positional_intersect(p1, p2, candidates):
  pip1 = content_pos_index[p1][1]
  pip2 = content_pos_index[p2][1]
  res = []
  for doc1, indexes1 in pip1.items():
    for doc2, indexes2 in pip2.items():
      if int(doc2) > int(doc1):
        break
      elif doc1 == doc2 and doc1 in candidates:
        for term1_index in indexes1:
          for term2_index in indexes2:
            if term1_index == term2_index-1:
              if not(doc1 in res):
                res.append(doc1)
            elif term1_index > term2_index:
              continue
            else:
              break
  return res

def phrase_query(query):
  query_words = preprocess(query)

  candidates = [c for c in content_pos_index[query_words[0]][1]]

  for i in range(1, len(query_words)):
    candidates = positional_intersect(query_words[i-1], query_words[i], candidates)

  return candidates


In [29]:
query = "تحریم‌های آمریکا علیه ایران"

print('boolean query:')
for doc in boolean_query(query)[:5]:
  print(data[doc]['url'])
print()
print('phrase query:')
for doc in phrase_query(query):
  print(data[doc]['url'])

boolean query:
https://www.farsnews.ir/news/14001222000450/توضیحات-یک-منبع-آگاه-درباره-وقفه-مذاکرات-وین
https://www.farsnews.ir/news/14000908000357/هتل-کوبورگ-زیر-ذره‌بین-بهارستان-تاکید-وکلای-ملت-بر-انتفاع-اقتصادی
https://www.farsnews.ir/news/14000819000731/مذاکره‌کنندگان-مراقب-زنجیره-تحریم‌ها-باشند-حداقل-4-تا-6-ماه-برای
https://www.farsnews.ir/news/14000821000097/راهبرد-مذاکره-هسته‌ای-نداریم-چگونه-تأمین-کننده-منافع-کشور-خواهد-بود-
https://www.farsnews.ir/news/14000803000676/اهرم‌سازی-از-افغانستان-در-برجام-نقطه-عزیمت-آمریکا-در-مذاکرات-جامع-با

phrase query:
https://www.farsnews.ir/news/14001222000450/توضیحات-یک-منبع-آگاه-درباره-وقفه-مذاکرات-وین
https://www.farsnews.ir/news/14001024000193/نباید-مانند-دولت-گذشته-در-مذاکرات-افراط-کرد-توافق-موقت-راهبرد-اصلی


In [39]:
import re

def process_query(query):
  phrase_queries = re.findall('"([^"]*)"', query)
  bool_query = re.sub('"([^"]*)"', '', query)


  candidates = []
  if len(bool_query) != 0:
    candidates = [boolean_query(bool_query)]

  if len(phrase_queries) == 0:
    return candidates[0]

  for phrase in phrase_queries:
    candidates.append(phrase_query(phrase))

  return candidates_intersection(candidates)

In [46]:
# process_query('"تحریم هسته‌ای" آمریکا ! ایران')
print_result(process_query('"تحریم هسته‌ای"'))

[['6929', '7330', '9207', '9360', '9694']]
['توضیحات', 'منبع', 'آگاه', 'وقفه', 'مذاکرات', 'وین'] https://www.farsnews.ir/news/14001222000450/توضیحات-یک-منبع-آگاه-درباره-وقفه-مذاکرات-وین ----------------------------
['بیانیه', 'اتحادیه', 'دانشجو', 'مستقل|', '۴', 'مطالبه', 'مذاکره', 'هسته\u200cای'] https://www.farsnews.ir/news/14001209001193/بیانیه-اتحادیه-دانشجویان-مستقل|-۴-مطالبه-از-مذاکره-کنندگان-هسته‌ای ----------------------------
['احیا', 'برجا', 'به\u200cتنهایی', 'گره', 'مشکلات', 'کشور', 'برطرف', 'کرد#کن'] https://www.farsnews.ir/news/14001011000132/احیای-برجام-به‌تنهایی-هیچ-گرهی-از-مشکلات-کشور-را-برطرف-نمی‌کند ----------------------------
['۹', 'دی', 'روز', 'فراموش', 'نشدن', 'تاریخ', 'انقلاب', 'اسلام'] https://www.farsnews.ir/news/14001007000363/۹-دی-یکی-از-روزهای-فراموش-نشدنی-در-تاریخ-انقلاب-اسلامی-است ----------------------------
['آمریکا', '«', 'راستی\u200cآزمایی', '»', 'لغو', 'تحریم', 'مشکل', 'دارد/', 'اطلاع', 'روحانی', 'ماجرا', 'افزایش', 'قیمت', 'بنزین'] https://www.farsnews